<div dir='rtl'>
<h1>پروژه سوم</h1>
</div>

<div dir='rtl'>
<h2>پیاده‌سازی</h2>
</div>

In [2]:
import turtle
import math
import random
from time import sleep
from copy import deepcopy
from enum import Enum
from typing import Union
from sys import argv
from timeit import default_timer as timer

In [3]:
VERTICES_COUNT = 6
SLEEP_TIME = 1.5

Line = tuple[int, int]

In [4]:
class Player(Enum):
    RED = 'red'
    BLUE = 'blue'

    def __invert__(self) -> 'Player':
        return Player.RED if self == Player.BLUE else Player.BLUE

<div dir='rtl'>
کلاس
Player
که نوعی 
enum
است، برای حذف کد تکراری از کلاس
Sim
اضافه شده است.
</div>

In [5]:
class MinimaxType(Enum):
    MIN = 0
    MAX = 1

    def __invert__(self) -> 'MinimaxType':
        return MinimaxType.MIN if self == MinimaxType.MAX else MinimaxType.MAX

In [6]:
def winner(player_moves: dict[Player, list[Line]]) -> tuple[Union[Player, None], Union[tuple[Line, Line, Line], None]]:
    for color, moves in player_moves.items():
        if len(moves) < 3:
            continue
        for i in range(0, len(moves) - 2):
            for j in range(i + 1, len(moves) - 1):
                for k in range(j + 1, len(moves)):
                    if len(set(moves[i] + moves[j] + moves[k])) == 3:
                        return ~color, (moves[i], moves[j], moves[k])
    return None, None

<div dir='rtl'>
تابع فوق به ازای هر 3 خطی که هر بازیکن گذاشته است، تعداد رئوس تشکیل‌دهنده این 3 خط را بدست می‌آورد و در صورتی که این مقدار برابر با 3 باشد، آن را یک مثلث به حساب آورده و برنده بازی را مشخص می‌کند.
</div>

In [18]:
class SimGui:
    Dot = tuple[float, float]

    def __init__(self, title: str, width: int, height: int, vertices_count: int):
        self._screen = turtle.Screen()
        self._screen.setup(width, height)
        self._screen.title(title)
        self._screen.bgcolor(0.117, 0.117, 0.117)
        self._screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self._screen.tracer(0, 0)
        turtle.hideturtle()
        self._vertices_count = vertices_count
        self._gen_dots()
        self._draw_board()

    def _gen_dots(self) -> None:
        self._dots = []
        for angle in range(0, 360, 360 // self._vertices_count):
            self._dots.append((math.cos(math.radians(angle)),
                               math.sin(math.radians(angle))))

    def _draw_dot(self, x: float, y: float, color: str) -> None:
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def _draw_line(self, p1: Dot, p2: Dot, color: str, pensize: int = 6) -> None:
        turtle.up()
        turtle.pensize(pensize)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def _draw_board(self) -> None:
        for dot in self._dots:
            self._draw_dot(dot[0], dot[1], 'dark gray')

    def _lineToDots(self, line: Line) -> tuple[Dot, Dot]:
        return ((math.cos(math.radians(line[0] * 360 // self._vertices_count)),
                 math.sin(math.radians(line[0] * 360 // self._vertices_count))),
                (math.cos(math.radians(line[1] * 360 // self._vertices_count)),
                 math.sin(math.radians(line[1] * 360 // self._vertices_count))))

    def draw(self, player_moves: dict[Player, list[Line]] = {}) -> None:
        turtle.clear()
        self._draw_board()
        for color, moves in player_moves.items():
            for move in moves:
                self._draw_line(*self._lineToDots(move), color.value)
        self._screen.update()

    def show_triangle(self, triangle: tuple[Line, Line, Line]) -> None:
        for line in triangle:
            self._draw_line(*self._lineToDots(line), 'white', 3)
        self._screen.update()

    def close(self) -> None:
        self._screen.bye()

<div dir='rtl'>
تابع 
show_triangle
در این کلاس، برای نشان دادن مثلثی که باعث باخت یک بازیکن شده است، اضافه شده است. این تابع مثلث ساخته شده را به رنگ سفید درمی‌آورد.
</div>

In [8]:
class MinimaxNode:
    _type: MinimaxType
    _value: float
    _children: dict[Line, 'MinimaxNode']
    _parent: Union['MinimaxNode', None]
    _move: Union[Line, None]
    _player: Player
    _depth: int
    _max_depth: int
    _prune: bool
    _alpha: float
    _beta: float
    _available_moves: list[Line]
    _player_moves: dict[Player, list[Line]]

    def __init__(self, parent: Union['MinimaxNode', None] = None, available_moves: list[Line] = [],
                 player_moves: dict[Player, list[Line]] = {}, prune: bool = False, max_depth: int = 1):
        self._parent = parent
        self._move = None
        self._children = {}
        self._value = 0
        self._type = MinimaxType.MAX if not parent else ~parent._type
        self._depth = 0 if not parent else parent._depth + 1
        self._max_depth = parent._max_depth if parent else max_depth
        self._player = Player.RED if not parent else ~parent._player
        self._alpha = -math.inf if not parent else parent._alpha
        self._beta = math.inf if not parent else parent._beta
        self._available_moves = deepcopy(parent._available_moves) if parent else deepcopy(available_moves)
        self._player_moves = {player: deepcopy(moves) for player, moves in parent._player_moves.items()} if parent else \
            {player: deepcopy(moves) for player, moves in player_moves.items()}
        self._prune = parent._prune if parent else prune

    def _evaluate(self) -> float:
        w = winner(self._player_moves)[0]
        if w is not None:
            return math.inf if w == Player.RED else -math.inf
        h = 0
        for move in self._available_moves:
            res = winner({self._player: self._player_moves[self._player] + [move]})[0]
            if res:
                h -= 1 # player is losing
            res = winner({~self._player: self._player_moves[~self._player] + [move]})[0]
            if res:
                h += 1 # player is winning
        return h if self._player == Player.RED else -h

    def _minimax(self) -> tuple[float, int]:
        if winner(self._player_moves)[0] is not None or self._depth == self._max_depth:
            self._value = self._evaluate()
            return self._value, self._depth
        self._value = -math.inf if self._type == MinimaxType.MAX else math.inf
        optimal_depth = 0
        for move in deepcopy(self._available_moves):
            self._available_moves.remove(move)
            self._player_moves[self._player].append(move)
            child = MinimaxNode(self)
            self._children[move] = child
            val, dep = child._minimax()

            if self._type == MinimaxType.MAX:
                if val > self._value:
                    self._value = val
                    self._move = move
                    optimal_depth = dep
                elif val == self._value and dep > optimal_depth:
                    self._move = move
                    optimal_depth = dep
                self._alpha = max(self._alpha, self._value)
            else:
                if val < self._value:
                    self._value = val
                    self._move = move
                    optimal_depth = dep
                elif val == self._value and dep > optimal_depth:
                    self._move = move
                    optimal_depth = dep
                self._beta = min(self._beta, self._value)

            if self._prune and self._alpha >= self._beta:
                break

            self._player_moves[self._player].remove(move)
            self._available_moves.append(move)

        return self._value, optimal_depth

    def get_best_move(self) -> Line:
        self._minimax()
        return self._move  # type: ignore

<div dir='rtl'>
برای انجام الگوریتم
Minimax
از کلاس
MinimaxNode
استفاده شده که نتیجه آن، ساخت درخت
Minimax
است.<br/>
<h3>هیوریستیک</h3>
برای هیوریستیک باید احتمال تشکیل مثلث و باخت هر بازیکن را در نظر بگیریم. به همین دلیل، ابتدا مقدار هیوریستیک را برابر با صفر می‌گذاریم و سپس به ازای هر خط در
available_moves،
اگر بازیکن با انتخاب این خط یک مثلث تشکیل می‌داد، یک واحد از مقدار هیوریستیک کم می‌کنیم و اگر حریف با انتخاب این خط باعث تشکیل مثلث می‌شد، یک واحد به هیوریستیک اضافه می‌کنیم. در نهایت اگر بازیکن مرحله فعلی قرمز بود، مقدار هیوریستیک، و اگر آبی بود، (استیت مینیمم‌کننده)، قرینه این مقدار را برمی‌گردانیم. لازم به ذکر است که اگر در استیت فعلی برنده و بازنده مشخص شده باشد، این تابع، اگر برنده قرمز باشد مقدار
&infin;
و اگر برنده آبی باشد، مقدار
&infin;-
را برمی‌گرداند.<br/><br/>
<b>‌پس از چند بار انجام الگوریتم مشخص شد که این الگوریتم در عمق 1 و 3 به خوبی عمل می‌کند ولی در عمق‌های 5 و 7، احتمال برد بسیار کاهش می‌یابد. پس از بررسی‌های مکرر متوجه شدم که مشکل این است که در عمق‌های بالا، برنده و بازنده در اکثر
nodeهای
درخت به صورت قطعی مشخص شده‌اند و در نتیجه مقادیر اکثر شاخه‌ها 
&infin;
و یا
&infin;-
خواهد بود. این
agent
با انجام الگوریتم
Minimax
فرض می‌کند که حریف نیز کاملا هوشمندانه عمل می‌کند در صورتی که حریف کاملا رندوم عمل می‌کند. همین دلیل باعث می‌شود که 
agent
زمانی که به یک
stateای
می‌رسد که مقدار تمامی شاخه‌های آن 
&infin;-
است، تفاوتی بین آن‌ها نبیند و یکی از آن‌ها را انتخاب کند. شاخه انتخاب شده می‌تواند باعث شود که بازیکن قرمز در مرحله‌ای ببازد که هنوز حرکت دیگری وجود دارد. این مورد در حالتی است که اگر 
agent
مسیر طولانی‌تری را انتخاب می‌کرد، ممکن بود با عملکرد رندوم بازیکن آبی به برد برسد. به همین دلیل تصمیم گرفتم عمق برگی که امتیازش به کمک تابع
evaluate_
محاسبه می‌شود را نیز در الگوریتم دخیل کنم که اگر در یک 
state
چند شاخه با مقدار یکسان قرار داشت، شاخه‌ای انتخاب شود که عمق برگی که به 
&infin;-
رسیده است بیشتر باشد. همین مورد باعث افزایش احتمال برد بازیکن قرمز در عمق‌های بالا می‌شود.</b>
</div>

In [9]:
class Sim:
    _gui: Union[SimGui, None]
    _turn: Player
    _player_moves: dict[Player, list[Line]]
    _available_moves: list[Line]
    _minimax_depth: int
    _prune: bool

    def __init__(self, minimax_depth: int, prune: bool, gui: bool):
        self._prune = prune
        self._minimax_depth = minimax_depth
        self._gui = SimGui('Game of Sim', 800, 800, VERTICES_COUNT) if gui else None

    def _initialize(self) -> None:
        self._available_moves = []
        for i in range(0, VERTICES_COUNT):
            for j in range(i, VERTICES_COUNT):
                if i != j:
                    self._available_moves.append((i, j))
        self._turn = random.choice([Player.RED, Player.BLUE])
        self._player_moves = {player: [] for player in Player}
        if self._gui:
            self._gui.draw()

    def _swap_turn(self) -> None:
        self._turn = ~self._turn

    def _minimax(self) -> tuple[Line, float]:
        root = MinimaxNode(available_moves=self._available_moves, player_moves=self._player_moves,
                           prune=self._prune, max_depth=self._minimax_depth)
        return root.get_best_move(), root._value

    def _enemy_move(self) -> Line:
        return random.choice(self._available_moves)

    def _game_over(self) -> tuple[Union[Player, None], Union[tuple[Line, Line, Line], None]]:
        return winner(self._player_moves)

    def play(self) -> Player:
        self._initialize()
        while True:
            selection = self._minimax()[0] if self._turn == Player.RED else self._enemy_move()
            selection = tuple(sorted(selection))

            if selection in self._player_moves.values():
                raise Exception("Duplicate Move!")

            self._player_moves[self._turn].append(selection)
            self._available_moves.remove(selection)
            self._swap_turn()
            if self._gui:
                self._gui.draw(self._player_moves)
                sleep(SLEEP_TIME)
            res = self._game_over()
            if res[0]:
                if self._gui:
                    self._gui.show_triangle(res[1])  # type: ignore
                    sleep(SLEEP_TIME)
                return res[0]

    def close(self) -> None:
        if self._gui:
            self._gui.close()

In [9]:
def calcWinChanceAndTime(depth: int, prune: bool, test_count: int) -> None:
    game = Sim(minimax_depth=depth, prune=prune, gui=False)
    start = timer()
    result = {p: 0 for p in Player}
    for i in range(test_count):
        print(f"Processing Test {i + 1}/{test_count}", end="\r")
        res = game.play()
        result[res] += 1
    end = timer()
    print(result)
    print(f"Depth: {depth}, Prune: {prune}")
    print(f"Time: {(end - start) / test_count:.4f}s")
    print(f"Win chance: {result[Player.RED] * 100 // test_count}%")
    print()

<div dir='rtl'>
<h2>بررسی نتایج</h2>
</div>

In [20]:
game = Sim(minimax_depth=5, prune=True, gui=True)
res = game.play()
print("Winner: ", res)
game.close()

Winner:  Player.RED


In [18]:
calcWinChanceAndTime(1, False, 100)
calcWinChanceAndTime(3, False, 100)
calcWinChanceAndTime(5, False, 50)

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 1, Prune: False
Time: 0.0047s
Win chance: 99%

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 3, Prune: False
Time: 0.4792s
Win chance: 99%

{<Player.RED: 'red'>: 50, <Player.BLUE: 'blue'>: 0}
Depth: 5, Prune: False
Time: 45.8823s
Win chance: 100%



<div dir='rtl'>
در این بخش چون زمان انجام الگوریتم با عمق 5 و بدون هرس بسیار زیاد است، این یک مورد را به جای 100 بار، 50 بار اجرا کردم.
</div>

In [19]:
calcWinChanceAndTime(1, True, 100)
calcWinChanceAndTime(3, True, 100)
calcWinChanceAndTime(5, True, 100)
calcWinChanceAndTime(7, True, 100)

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 1, Prune: True
Time: 0.0047s
Win chance: 99%

{<Player.RED: 'red'>: 98, <Player.BLUE: 'blue'>: 2}
Depth: 3, Prune: True
Time: 0.0894s
Win chance: 98%

{<Player.RED: 'red'>: 100, <Player.BLUE: 'blue'>: 0}
Depth: 5, Prune: True
Time: 1.1692s
Win chance: 100%

{<Player.RED: 'red'>: 100, <Player.BLUE: 'blue'>: 0}
Depth: 7, Prune: True
Time: 12.4892s
Win chance: 100%



<div dir='rtl'>
همانطور که مشاهده می‌شود، زمان اجرای تمامی حالات به مراتب نسبت به حالتی که هرس را انجام نمی‌دادیم، کاهش یافته است.
</div>

<div dir='rtl'>
احتمال برد در تمامی حالات بالای 98 درصد است که نشان‌دهنده عملکرد مناسب الگوریتم و هیوریستیک مورد استفاده است.
</div>

<div dir='rtl'>
<h2>سوالات</h2>
</div>

<div dir='rtl'>
<ol>
    <li>یک هیوریستیک خوب باید بتواند با احتمال بالایی، برد یا باخت بازیکن را پیش‌بینی کند که بتوانیم بهترین مسیر را از ابتدا پیدا کنیم. در این سوال برنده و بازنده با تشکیل مثلث مشخص می‌شوند و به همین دلیل بهترین روش این است که روی احتمال تشکیل مثلث توسط هر بازیکن مانور بدهیم. این هیوریستیک به ازای هر مثلثی که حریف بتواند در حرکت بعدی‌اش تشکیل دهد یک امتیاز مثبت، و به ازای هر مثلثی که بازیکن فعلی بتواند در حرکت بعدی‌اش تشکیل دهد، یک امتیاز منفی در نظر می‌گیرد. و به همین دلیل باعث می‌شود حالتی را انتخاب کنیم که احتمال بازنده بودن خودمان (برنده شدن حریف) کمینه شود. یک روش دیگر این است که مثلث‌های قابل تشکیل توسط حریف را محاسبه نکنیم و فقط به ازای تشکیل هر مثلث توسط خودمان، یک امتیاز منفی در نظر بگیریم. با امتحان کردن این هیوریستیک متوجه می‌شویم که احتمال برد کاهش می‌یابد. روش دیگر این است که مجموعه درجه رئوس را محاسبه کنیم که در این حالت اگر دو خط کاملا با فاصله از همدیگر قرار بگیرند، مقدار هیوریستیک برابر با 4 خواهد شد و اگر دو خط مورد نظر در یک راس مشترک باشند نیز، مقدار هیوریستیک برابر با 4 خواهد بود در صورتی که در حالت دوم، احتمال تشکیل مثلث بیشتر است. در همین مثال، هیوریستیک پیاده‌سازی شده برای حالت اول مقدار 0 و برای حالت دوم مقدار 1 را برمی‌گرداند.</li><br/>
    <li>وقتی عمق افزایش می‌یابد، بدیهی‌ست که تعداد گره‌های مشاهده شده افزایش پیدا می‌کند و در نتیجه زمان الگوریتم نیز بیشتر می‌شود. برای مثال اگر بخواهیم تا عمق 
    n-ام
    را بدون هرس کردن محاسبه کنیم، تعداد گره‌های مشاهده شده به صورت زیر بدست می‌آید. همچنین، به دلیل اینکه با افزایش عمق احتمال اینکه برنده و بازنده بازی به صورت قطعی مشخص شود بیشتر می‌شود، به صورت دقیق‌تری می‌توانیم مسیر مناسب را پیش‌بینی کنیم.
    همانطور که پیش‌تر گفته شد، اگر عمق برگ‌ها را در الگوریتم دخیل نکنیم، شانس پیروزی کاهش می‌یابد، اما با دخیل کردن این مورد، احتمال برد به مقدار زیادی افزایش می‌یابد.</li><br/>
    <li>زمانی که از هرس کردن استفاده می‌کنیم، ترتیب ساخت درخت اهمیت زیادی دارد زیرا ممکن است در یک ترتیب تمام 
    stateها
    مانند حالتی که هرس نمی‌کنیم پیمایش شوند و در ترتیب دیگری، ممکن است تعداد زیادی از
    stateها 
    هرس شوند و باعث کاهش زمان انجام الگوریتم شود. در این سوال ترتیب اولیه اهمیتی ندارد زیرا تمام رئوس و خط‌ها یکسان هستند و هیچ اختلافی بین هیچ جفتی از آن‌ها وجود ندارد. اما لازم است که در تمام مراحل همین ترتیب را حفظ کنیم زیرا طبق الگوریتم، اگر چند حالت با امتیاز مشابه وجود داشته باشد، همواره اولین حالت انتخاب می‌شود و در صورتی که ترتیب اولیه حفظ شود، اکثر یال‌ها از رئوس با شماره کوچک‌تر خارج می‌شود که باعث نزدیک شدن این رئوس به تشکیل مثلث می‌شود. به همین دلیل در هرس آلفا و بتا نیز ممکن است قبل از پیمایش کل درخت، مسیر مناسب را پیدا کنیم و نیاز به ساخت کل درخت نباشد.</li>
</ol>
</div>

$$statesCount = \dfrac{(15 - {currentDepth})!}{(15 - {currentDepth} - n)!}$$
$$if\ n = 7\ \&\ currentDepth = 0\ (root)\longrightarrow statesCount = \dfrac{15!}{8!} = 32,432,400$$